In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnableBranch
from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain_community.chat_message_histories import ChatMessageHistory

from dotenv import load_dotenv
import requests
import os

load_dotenv()

API_KEY = os.getenv('GOOGLE_API_KEY')

In [2]:
# Google gemini for chat based interaction
chat_model = ChatGoogleGenerativeAI(model='gemini-pro', temperature=0.2, google_api_key=API_KEY,
                                    convert_system_message_to_human=True)

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, PromptTemplate

# Prompt template for guiding the all chat conversation
system_message = """You are a helpful assistant. Answer all questions to the best of your ability with detailed and 
                    actionable response."""
system_prompt = ChatPromptTemplate.from_messages([('system', system_message), 
                                           MessagesPlaceholder(variable_name='messages')])

# Chain the model with the 
chain = system_prompt | chat_model

The `MessagesPlaceholder` above inserts chat messages passed into the chain’s input.

In [4]:
from langchain_core.messages import HumanMessage, AIMessage

prompt = 'Translate this sentence from English to French: I love programming.'
chain.invoke(
    {'messages': [HumanMessage(content=prompt)]}
)

AIMessage(content=" J'aime la programmation.")

That look greats, however, the model itself doesn't have the ability to sustain a normal conversation. Just see that,

In [5]:
chain.invoke({'messages': [HumanMessage(content='What did you just say')]})

AIMessage(content='I am a helpful assistant designed to provide information and answer questions to the best of my abilities. I am still under development, but I am learning new things every day.\n\nIf you have a question, please feel free to ask me. I will do my best to answer it.')

You can see that he doesn't have any conception about the last conversation, and seem to be lost.

Having a message history will fix this issue and allow him to track the previous conversation through history.    

# Memory without RAG

In [6]:
# Based on preference
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.schema import SystemMessage


In [7]:
# Prompt template for guiding the all chat conversation
system_message = "You are a helpful assistant. Answer all questions to the best of your ability."
system_prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content=system_message), 
        MessagesPlaceholder(variable_name='chat_history'),
        HumanMessagePromptTemplate.from_template("{human_input}")
    ])

In [8]:
from langchain.chains import LLMChain
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation = LLMChain(llm=chat_model, prompt=system_prompt, memory=memory, verbose=False)

In [9]:
prompt = 'Translate this sentence from English to French: I love programming.'
conversation.predict(human_input=prompt)

"J'adore la programmation."

In [10]:
conversation.predict(human_input='What did you just say')

'I just said "I love programming" in French.\n\nThe original sentence in English was "I love programming." I used Google Translate to translate this sentence into French, which gave me the following result: "J\'adore la programmation."\n\nI hope this is helpful! Let me know if you have any other questions.'

That look awesome, we have just introduce a memory to the chatbot. Now we can have a great conversation with it which allow us to use it in various application like assistant, client support or ecommerce. Also, we can use it by adding external data source to interact with which bring us to the next step **Retrieval**.

# Retrieval

In this section, we will use Milvus as VectorStore to store all the information that the chatbot through all of the conversation. 

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Milvus
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from pymilvus import connections, db, utility

In [12]:
# Load new data
loader = PyPDFLoader("Selling_the_Invisible_A_Field_Guide_to_M.pdf")
docs = loader.load()

In [13]:
# Embeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=API_KEY)

# Document Splitter
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100
    )
docs_splitted = text_splitter.split_documents(docs)

# Vectore store
connection_args = {"host": "127.0.0.1", "port": "19530"}
vector_store = Milvus(embeddings, connection_args=connection_args, 
                      collection_name='Selling_Invisible')

# k is the number of chunks to retrieve
retriever = vector_store.as_retriever()

In [14]:
from langchain.chains.combine_documents import create_stuff_documents_chain

SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context.

<context>
{context}
</context>
""" 

question_answering_prompt = ChatPromptTemplate.from_messages(
    [("system", SYSTEM_TEMPLATE), MessagesPlaceholder(variable_name="messages")]
)
document_chain = create_stuff_documents_chain(chat_model, question_answering_prompt)

In [15]:
document_chain.invoke({
    "context": retriever.invoke("How to sell a service ?"),
    "messages": [HumanMessage(content="How to sell a service ?")]
})

'The context does not provide information on how to sell a service.'

Assembling stuff.

In [16]:
from langchain_core.runnables import RunnablePassthrough
from typing import Dict

def parse_retriever_input(params: Dict):
    return params["messages"][-1].content

retrieval_chains = RunnablePassthrough.assign(context=parse_retriever_input | retriever,).assign(answer=document_chain)

In [17]:
retrieval_chains.invoke({
    "messages": [HumanMessage(content="How to sell a service ?")]
})

{'messages': [HumanMessage(content='How to sell a service ?')],
 'context': [Document(page_content='• Tell people in a single sentence why they should buy from you. \n• Advertise. \n• Make your service easy to order or buy. \n• Talk about the other person, not yourself. \n• Show that you care passionately about your clients’ business. \n• Write a mission statement and keep it private. \n• Draw a clear map. After every missi on statement, add an objectives statement. \n• If the mission statement doesn’t inspire people to act, change it.', metadata={'source': 'Selling_the_Invisible_A_Field_Guide_to_M.pdf', 'page': 8}),
  Document(page_content='are buying an experience. \n \nIf you are selling a service, you’re  actually selling a relationship. \n \nBefore you try to satisfy your client, understand and satisfy the person. That means knowing his \nlikes, hobbies, pets, family life, dreams and goals. \n \nYour real competition is sitting across the table from you. Plan accordingly. \n \nYou

In [18]:
retriever.invoke("Tell me more!")

[Document(page_content='Last word: Take the risk. Get out there and let opportunity hit you.', metadata={'source': 'Selling_the_Invisible_A_Field_Guide_to_M.pdf', 'page': 9}),
 Document(page_content='subscriptions if reply forms are ready and postage has been paid for. Forget looking \nlike the superior choice, focus on being an ex cellent choice. Eliminate anything that \nmakes you a bad choice.', metadata={'source': 'Selling_the_Invisible_A_Field_Guide_to_M.pdf', 'page': 5}),
 Document(page_content='• Tell people in a single sentence why they should buy from you. \n• Advertise. \n• Make your service easy to order or buy. \n• Talk about the other person, not yourself. \n• Show that you care passionately about your clients’ business. \n• Write a mission statement and keep it private. \n• Draw a clear map. After every missi on statement, add an objectives statement. \n• If the mission statement doesn’t inspire people to act, change it.', metadata={'source': 'Selling_the_Invisible_A_Fiel

## Query transformation

In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

QUERY_SYSTEM_TEMPLATE = """Answer the user's questions based on the below context. """ 
query_transform_prompt = ChatPromptTemplate.from_messages([
    ("system", QUERY_SYSTEM_TEMPLATE),
    MessagesPlaceholder(variable_name="messages")
])

query_transforming_retriever_chain = RunnableBranch(
    (
        lambda x: len(x.get("messages", [])) == 1,
        (lambda x: x["messages"][-1].content) | retriever
    ),
    query_transform_prompt | chat_model | StrOutputParser() | retriever
).with_config(run_name="chat_retriever_chain")

In [20]:
query_transform_chain = query_transform_prompt | chat_model

query_transform_chain.invoke({
    "messages": [
        HumanMessage(content="How to sell a service ?"),
        AIMessage(content='• Tell people in a single sentence why they should buy from you. \n• Advertise. \n• Make your service easy to order or buy. \n• Talk about the other person, not yourself. \n• Show that you care passionately about your clients’ business.'),
        HumanMessage(content="Summarize!")
    ]
})

AIMessage(content="To sell a service, focus on the customer's needs and make it easy for them to buy from you.")

In [21]:
conversation_retriever_chain = RunnablePassthrough.assign(
    context=query_transforming_retriever_chain
).assign(answer=document_chain)

In [22]:
conversation_retriever_chain.invoke(
    {
        "messages": [HumanMessage(content="How to sell a service ?")]
    }
)

{'messages': [HumanMessage(content='How to sell a service ?')],
 'context': [Document(page_content='• Tell people in a single sentence why they should buy from you. \n• Advertise. \n• Make your service easy to order or buy. \n• Talk about the other person, not yourself. \n• Show that you care passionately about your clients’ business. \n• Write a mission statement and keep it private. \n• Draw a clear map. After every missi on statement, add an objectives statement. \n• If the mission statement doesn’t inspire people to act, change it.', metadata={'source': 'Selling_the_Invisible_A_Field_Guide_to_M.pdf', 'page': 8}),
  Document(page_content='are buying an experience. \n \nIf you are selling a service, you’re  actually selling a relationship. \n \nBefore you try to satisfy your client, understand and satisfy the person. That means knowing his \nlikes, hobbies, pets, family life, dreams and goals. \n \nYour real competition is sitting across the table from you. Plan accordingly. \n \nYou

In [23]:
conversation_retriever_chain.invoke(
    {
        "messages": [
            HumanMessage(content="How to sell a service ?"),
            AIMessage(
                content="- Tell people in a single sentence why they should buy from you.\n- Advertise.\n- Make your service easy to order or buy.\n- Talk about the other person, not yourself.\n- Show that you care passionately about your clients’ business."
            ),
            HumanMessage(content="Tell me more!"),
        ],
    }
)

{'messages': [HumanMessage(content='How to sell a service ?'),
  AIMessage(content='- Tell people in a single sentence why they should buy from you.\n- Advertise.\n- Make your service easy to order or buy.\n- Talk about the other person, not yourself.\n- Show that you care passionately about your clients’ business.'),
  HumanMessage(content='Tell me more!')],
 'context': [Document(page_content='• Tell people in a single sentence why they should buy from you. \n• Advertise. \n• Make your service easy to order or buy. \n• Talk about the other person, not yourself. \n• Show that you care passionately about your clients’ business. \n• Write a mission statement and keep it private. \n• Draw a clear map. After every missi on statement, add an objectives statement. \n• If the mission statement doesn’t inspire people to act, change it.', metadata={'source': 'Selling_the_Invisible_A_Field_Guide_to_M.pdf', 'page': 8}),
  Document(page_content='are buying an experience. \n \nIf you are selling a 

## Add memory to query transformation

In [24]:
chat_history = ChatMessageHistory()
ASSISTANT_PROMPT = "You're a helpful assistant that help human to reach their goal"
query_conversation_prompt = ChatPromptTemplate.from_messages([
    ("system", QUERY_SYSTEM_TEMPLATE),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

query_transforming_chain = query_conversation_prompt | chat_model
query_transforming_chain_with_history = RunnableWithMessageHistory(
    query_transforming_chain,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

In [25]:
query_transforming_retriever_chain = RunnableBranch(
    (
        lambda x: len(x.get("chat_history", [])) == 1,
        (lambda x: x["chat_history"][-1].content) | retriever
    ),
    query_transforming_chain_with_history | StrOutputParser() | retriever
).with_config(run_name="chat_mode")

In [26]:
SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context.

<context>
{context}
</context>
""" 

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_TEMPLATE), 
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}")
    ]
)

document_chain = question_answering_prompt | chat_model

# Add history
document_chain_with_history = RunnableWithMessageHistory(
    document_chain,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

In [27]:
conversation_retriever_chain = RunnablePassthrough.assign(
    context=query_transforming_retriever_chain
).assign(answer=document_chain_with_history)

In [28]:
response = conversation_retriever_chain.invoke(
    {
        "input": "How to use social proof for selling my stuff ?"
    },
    {"configurable": {"session_id": "unused"}}
)
print(response['answer'].content)

**Use testimonials and reviews:**

* Display positive testimonials and reviews from satisfied customers on your website, social media pages, and marketing materials.
* Encourage customers to leave reviews on third-party platforms like Google My Business, Yelp, and Trustpilot.

**Showcase case studies and success stories:**

* Create case studies that highlight how your product or service has helped specific customers achieve their goals.
* Share success stories from your customers on your website, blog, and social media channels.

**Leverage social media:**

* Encourage customers to share their experiences with your product or service on social media.
* Run social media campaigns that encourage user-generated content, such as photo contests or testimonial videos.
* Partner with influencers in your industry to promote your product or service and generate social proof.

**Display social media proof:**

* Show the number of followers, likes, and shares your social media accounts have on y

In [29]:
response = conversation_retriever_chain.invoke(
    {
        "input": "Could you provide an example ?"
    },
    {"configurable": {"session_id": "unused"}}
)
print(response['answer'].content)

**Example of using social proof to sell a service:**

**Service:** ABC consulting services

**Social proof:**

* **Testimonials:** "ABC consulting services helped me increase my sales by 20% in just 6 months. I highly recommend their services to any business looking to grow." - John Smith, CEO of XYZ Company
* **Reviews:** "ABC consulting services is the best consulting firm I've ever worked with. They are knowledgeable, experienced, and results-oriented." - Mary Jones, Marketing Manager at ABC Company
* **Case study:** "How ABC consulting services helped XYZ Company achieve their business goals." - Case study available on the ABC consulting services website
* **Social media proof:** ABC consulting services has over 50,000 followers on LinkedIn and has been featured in several industry publications.

**How the social proof is used:**

* The testimonials and reviews are displayed prominently on the ABC consulting services website and marketing materials.
* The case study is featured on 

In [30]:
response

{'input': 'Could you provide an example ?',
 'context': [Document(page_content='• Tell people in a single sentence why they should buy from you. \n• Advertise. \n• Make your service easy to order or buy. \n• Talk about the other person, not yourself. \n• Show that you care passionately about your clients’ business. \n• Write a mission statement and keep it private. \n• Draw a clear map. After every missi on statement, add an objectives statement. \n• If the mission statement doesn’t inspire people to act, change it.', metadata={'source': 'Selling_the_Invisible_A_Field_Guide_to_M.pdf', 'page': 8}),
  Document(page_content='should buy from you. \n• One good story beats a dozen adjectives. Stories are the easiest way to spread word of \nmouth. Stories are passed on and make things more credible or personal and more \npersuasive. \n• Attack the stereotype your  prospect has about you. \n• Good basic communication is good basic marketi ng.  Be specific, and vivid. Use familiar \nexamples to

# Add memory management to RAG

## RAG without Memory

In [31]:
from langchain import hub


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

question_answering_prompt = ChatPromptTemplate.from_messages(
    [("system", SYSTEM_TEMPLATE), MessagesPlaceholder(variable_name="messages")]
)
prompt = """Use the following context to answer the question at the end.
    Always say “Thanks for asking!” » at the end of the answer.

    {context}
    """

QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt)
question_answering_prompt = ChatPromptTemplate.from_messages([
    ("system", prompt),
    # MessagesPlaceholder(variable_name="messages")
    ("human", "{question}")
])

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | question_answering_prompt
    | chat_model
    | StrOutputParser()
)

In [32]:
rag_chain.invoke("How to sell a service ?")

'You’re actually selling a relationship when you’re selling a service. Thanks for asking!'

Show source that is used to understand well the response.  

In [33]:
from langchain_core.runnables import RunnableParallel

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x['context'])))
    | QA_CHAIN_PROMPT
    | chat_model
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [34]:
rag_chain_with_source.invoke("How to sell a service ?")

{'context': [Document(page_content='• Tell people in a single sentence why they should buy from you. \n• Advertise. \n• Make your service easy to order or buy. \n• Talk about the other person, not yourself. \n• Show that you care passionately about your clients’ business. \n• Write a mission statement and keep it private. \n• Draw a clear map. After every missi on statement, add an objectives statement. \n• If the mission statement doesn’t inspire people to act, change it.', metadata={'source': 'Selling_the_Invisible_A_Field_Guide_to_M.pdf', 'page': 8}),
  Document(page_content='are buying an experience. \n \nIf you are selling a service, you’re  actually selling a relationship. \n \nBefore you try to satisfy your client, understand and satisfy the person. That means knowing his \nlikes, hobbies, pets, family life, dreams and goals. \n \nYour real competition is sitting across the table from you. Plan accordingly. \n \nYour prospect faces three options: using your servic e, doing it th

## Add chat history

In [35]:
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
contextualize_q_chain = contextualize_q_prompt | chat_model | StrOutputParser()

In [36]:
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input['question']

rag_chain = (
    RunnablePassthrough.assign(
        context=contextualized_question | retriever | format_docs
    )
    # | RunnablePassthrough.assign(context=(lambda x: format_docs(x["chat_history"])))
    | qa_prompt
    | chat_model
)

In [37]:
chat_history = []
question = "How to sell a service ?"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg])

second_question = "Provide an example !"
rag_chain.invoke({"question": second_question, "chat_history": chat_history})

AIMessage(content='**Example:**\n\nA financial advisor who focuses on building relationships with their clients, understanding their financial goals and concerns, and providing personalized advice tailored to their specific needs. This approach creates a sense of trust and loyalty, making clients more likely to stay with the advisor for the long term and refer their friends and family.')

In [38]:
chat_history

[HumanMessage(content='How to sell a service ?'),
 AIMessage(content='To sell a service, you should focus on understanding and satisfying the person, not just the client. By creating a relationship with your clients, you can provide them with an experience that goes beyond their expectations. Make every employee a marketer by ensuring that every interaction with your clients is a positive one.')]

# AGENT

In [39]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import Tool
from langchain.agents import initialize_agent, create_json_chat_agent
from langchain.tools import BaseTool
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

import random

In [40]:
search = DuckDuckGoSearchRun()
chat_model = ChatGoogleGenerativeAI(model='gemini-pro', temperature=0.2, google_api_key=API_KEY,
                                    convert_system_message_to_human=True)
# defining a single tool
tools = [
    Tool(
        name = "search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    )
]

In [41]:
def meaning_of_life(input=""):
    return 'The meaning of life is 42 if rounded but is actually 42.17658'

life_tool = Tool(
    name='Meaning of Life',
    func= meaning_of_life,
    description="Useful for when you need to answer questions about the meaning of life. input should be MOL "
)

@tool
def random_num(input=""):
    return random.randint(0,5)
    
random_tool = Tool(
    name='Random number',
    func= random_num,
    description="Useful for when you need to get a random number. input should be 'random'"
)

In [42]:
tools = [search, random_tool, life_tool]

# conversational agent memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)


# create our agent
conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=chat_model,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory
)

E:\software\anaconda3\envs\ai\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [43]:
conversational_agent.invoke("What time is it in Antananarivo?")



> Entering new AgentExecutor chain...
```json
{
    "action": "duckduckgo_search",
    "action_input": "What time is it in Antananarivo?"
}
```
Observation: The current local time in Antananarivo is 1 minute behind apparent solar time. Antananarivo on the map. Antananarivo is the capital of Madagascar. Latitude: -18.91. Longitude: 47.54; Population: 1,350,000; Open Antananarivo in Google Maps. The 25 largest cities in Madagascar. Antananarivo time to your local time converter Antananarivo is the capital of Madagascar. Current local time in Antananarivo and DST dates in 2024. Local time--:--EAT . Antananarivo summer time (DST) in 2024. No known DST-adjustments for year 2024 . Antananarivo time zone ... Want to see the time in Antananarivo, Madagascar compared with your home? Choose a date and time then click "Submit" and we'll help you convert it from Antananarivo, Madagascar time to your time zone. 2024 Feb 10 at 12 (12 Noon) 00. Convert Time From Antananarivo, Madagascar to any time

{'input': 'What time is it in Antananarivo?',
 'chat_history': [],
 'output': 'The current local time in Antananarivo is 1 minute behind apparent solar time.'}

In [70]:
class OrderTool(BaseTool):
    name = "Get Order"
    description = """Useful for when you need to get a specific order. But the order 
    is not specified ask to the user what is the order ID. If the user does not provide order ID, ask for it."""

    def _run(self, id: str):
        try:
            # Send the GET request and store the response
            response = requests.get(f"http://localhost:8000/orders/{id}")
            
            # Check for successful response (status code 200)
            if response.status_code == 200:
                # Get the response content
                data = response.json()  # Assuming the response is JSON format
                print(data)  # Print the data
            else:
                print(f"Error: {response.status_code}")  # Print error message with status code
            return data   
        except requests.exceptions.RequestException as e:
          print(f"Error: {e}")
    
    def _arun(self, webpage: str):
        raise NotImplementedError("This tool does not support async")


class OrderIcompleteTool(BaseTool):
    name = "Ask Order ID"
    description = """Useful for when you need to get a specific order where the order ID is not provided.
    In this situation you should ask for Order ID."""

    def _run(self):
        pass
    
    def _arun(self, webpage: str):
        raise NotImplementedError("This tool does not support async")

tools = [OrderTool(), OrderIcompleteTool(), life_tool, random_tool]

In [66]:
# conversational agent memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)

# create our agent
conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=chat_model,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory,
    handle_parsing_errors=True
)

In [67]:
conversational_agent.invoke("I want a detail of my order")



> Entering new AgentExecutor chain...
Could not parse LLM output: ```json
{
    "action": "Ask Order ID",
    "action_input": null
}
```
Observation: Invalid or incomplete response
Thought:```json
{
    "action": "Final Answer",
    "action_input": "I'm sorry, I cannot answer your question without knowing the order ID. Please provide the order ID so I can look up the details for you."
}
```

> Finished chain.


{'input': 'I want a detail of my order',
 'chat_history': [],
 'output': "I'm sorry, I cannot answer your question without knowing the order ID. Please provide the order ID so I can look up the details for you."}

In [68]:
conversational_agent.invoke("It's 171-9198151-1101146")



> Entering new AgentExecutor chain...
```json
{
    "action": "Get Order",
    "action_input": "171-9198151-1101146"
}
```{'index': 1, 'OrderID': '171-9198151-1101146', 'Date': '04-30-22', 'Status': 'Shipped - Delivered to Buyer', 'Fulfilment': 'Merchant', 'SalesChannel': 'Amazon.in', 'ship_service_level': 'Standard', 'Style': 'JNE3781', 'SKU': 'JNE3781-KR-XXXL', 'Category': 'kurta', 'Size': '3XL', 'ASIN': 'B09K3WFS32', 'courier_status': 'Shipped', 'Qty': 1, 'currency': 'INR', 'Amount': 406.0, 'ship_city': 'BENGALURU', 'ship_state': 'KARNATAKA', 'ship_postal_code': 560085.0, 'ship_country': 'IN', 'promotion_ids': 'Amazon PLCC Free-Financing Universal Merchant AAT-WNKTBO3K27EJC,Amazon PLCC Free-Financing Universal Merchant AAT-QX3UCCJESKPA2,Amazon PLCC Free-Financing Universal Merchant AAT-5QQ7BIYYQEDN2,Amazon PLCC Free-Financing Universal Merchant AAT-DSJ2QRXXWXVMQ,Amazon PLCC Free-Financing Universal Merchant AAT-CXJHMC2YJUK76,Amazon PLCC Free-Financing Universal Merchant AAT-CC4FAV

{'input': "It's 171-9198151-1101146",
 'chat_history': [HumanMessage(content='I want a detail of my order'),
  AIMessage(content="I'm sorry, I cannot answer your question without knowing the order ID. Please provide the order ID so I can look up the details for you.")],
 'output': 'The order with ID 171-9198151-1101146 was shipped and delivered to the buyer on 04-30-22. The order was fulfilled by Easy Ship.'}

# Tool with Function call

In [48]:
from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent

In [74]:
tools = [search, OrderTool(), OrderIcompleteTool()]
prompt = hub.pull("hwchase17/structured-chat-agent")

# conversational agent memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)

agent = create_structured_chat_agent(chat_model, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, memory=memory, handle_parsing_errors=True
)
agent_executor.invoke({"input": "Sorry, but my colis isn't arrived yet?"})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Ask Order ID",
  "action_input": {}
}
```NoneAction:
```
{
  "action": "Final Answer",
  "action_input": "I'm sorry, but I cannot help you with that. I do not have access to your order information."
}
```

> Finished chain.


{'input': "Sorry, but my colis isn't arrived yet?",
 'chat_history': [],
 'output': "I'm sorry, but I cannot help you with that. I do not have access to your order information."}

In [75]:
agent_executor.invoke({"input": "Here it is : It's 171-9198151-1101146"})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Get Order",
  "action_input": {
    "id": "171-9198151-1101146"
  }
}
```{'index': 1, 'OrderID': '171-9198151-1101146', 'Date': '04-30-22', 'Status': 'Shipped - Delivered to Buyer', 'Fulfilment': 'Merchant', 'SalesChannel': 'Amazon.in', 'ship_service_level': 'Standard', 'Style': 'JNE3781', 'SKU': 'JNE3781-KR-XXXL', 'Category': 'kurta', 'Size': '3XL', 'ASIN': 'B09K3WFS32', 'courier_status': 'Shipped', 'Qty': 1, 'currency': 'INR', 'Amount': 406.0, 'ship_city': 'BENGALURU', 'ship_state': 'KARNATAKA', 'ship_postal_code': 560085.0, 'ship_country': 'IN', 'promotion_ids': 'Amazon PLCC Free-Financing Universal Merchant AAT-WNKTBO3K27EJC,Amazon PLCC Free-Financing Universal Merchant AAT-QX3UCCJESKPA2,Amazon PLCC Free-Financing Universal Merchant AAT-5QQ7BIYYQEDN2,Amazon PLCC Free-Financing Universal Merchant AAT-DSJ2QRXXWXVMQ,Amazon PLCC Free-Financing Universal Merchant AAT-CXJHMC2YJUK76,Amazon PLCC Free-Financing Universal Mer

{'input': "Here it is : It's 171-9198151-1101146",
 'chat_history': [HumanMessage(content="Sorry, but my colis isn't arrived yet?"),
  AIMessage(content="I'm sorry, but I cannot help you with that. I do not have access to your order information.")],
 'output': 'Your order has been delivered.'}

In [77]:
prompt.pretty_print()

================================ System Message ================================

Respond to the human as helpfully and accurately as possible. You have access to the following tools:

{tools}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_input": "Final response to human"
}

Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation

==============